In [2]:
#importing the required library

import requests
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import re
import openpyxl

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
#script for extracting artciles from websites using Beautiful Soup and Requests

#setting a user agent for acessing the webpages
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'} 

#I have uploaded the INPUT_DATA sheet on google drive in order to access all the links using loop instead of scraping each of them manually
input_data='https://drive.google.com/file/d/1x9l-mK_0wfNetqlZD_huOUWu1-egaC_m/view?usp=sharing'
input_data='https://drive.google.com/uc?id=' + input_data.split('/')[-2]

#parsing the INPUT DATA sheet as dataframe using pandas
data = pd.read_csv(input_data)

for i in range(len(data)):
    url=data.at[i,'URL'] #this variable will be used to acess URLs
    file_name=data.at[i,'URL_ID'] #this variable will be used to name the scraoed files

    #this line is for skipping URLs with URL_ID : 44,57 and 144 as they are not working
    if file_name==44 or file_name== 57 or file_name==144:
        continue
    else:
        html = requests.get(f'{url}', headers=headers)

        soup = BeautifulSoup(html.text, 'html.parser')     #parsing the web pages

        title = soup.find("h1", class_="entry-title").text #extracting titles of web pages

        paragraphs = [p.text for p in soup.find_all("p")]  #extracting each paragraph separately

        f = open(f'{file_name}.txt', 'w',encoding="utf-8") #creating a new file with name = URL_ID

        f.write(title + '\n\n')                            #adding title to the file

        for p in paragraphs:                               #starting a for loop to add all the paragraphs in the .txt file from the aticle
            f.write(p + '\n\n')

In [5]:
#adding custom stop words from .txt file
#Creating an empty list which will later be filled with stopwords provided in .txt files of stop words folder

stop_words_list = []

for i in range(1,8):
  with open(f'StopWords_{i}.txt', 'r',encoding='latin-1') as f:
   
# acessing items on each line of .txt files and adding them to a list of custom stop words
    for line in f: 
      lines=line.strip()
      stop_words_list.append(line.strip())

# creating a temporary list to access special entries, example: 'AFGHANI  | Afghanistan ' 
temp=[] 
for word in stop_words_list:
  if ' ' in word:
    a=word.split('|')
    for words in a:
      temp.append(words)
    stop_words_list.remove(word) #removing duplicates

stop_words_list=stop_words_list+temp


In [9]:
#using openpyxl library to create and update an excel sheet to load the analysis data

excel = openpyxl.Workbook() #creating a workbook
sheet = excel.active        #going to active sheet
sheet.title= 'Output_Data'  #naming the sheet
sheet.append(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
             'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
             'AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','SYLLABLE PER WORD', 'PERSONAL PRONOUNS',
             'AVG WORD LENGTH'])  #adding required columns to the sheet
input_data = 'https://drive.google.com/file/d/1x9l-mK_0wfNetqlZD_huOUWu1-egaC_m/view?usp=sharing' 
input_data = 'https://drive.google.com/uc?id=' + input_data.split('/')[-2]

for i in range(len(data)):
    url = data.at[i, 'URL']  # this variable will be used to acess URLs
    file_name = data.at[i, 'URL_ID']  # this variable will be used to name the scraoed files

    # parsing the INPUT DATA sheet as dataframe using pandas
    data = pd.read_csv(input_data)
    if file_name==44 or file_name==57 or file_name==144:
      continue
    else:

      def count_pn(str):                       #function to count the total personal pronouns using refular expression (regex)
          pattern = r'\b(I|we|my|ours|us)\b'
          regex = re.compile(pattern, re.IGNORECASE)
          return len(regex.findall(text))


      # Get the list of stop words in English
      stop_words = nltk.corpus.stopwords.words('english')

      with open(f'{file_name}.txt', 'r') as f:
          # Read the contents of the file
          text = f.read()
          sentences = nltk.sent_tokenize(text)
          tokens = nltk.word_tokenize(text)
          # w = [nltk.word_tokenize(sentence) for sentence in sentences]
          words_after_stopwords_list = []
          total_words = []
      for w in tokens:
          if w not in stop_words and w not in string.punctuation:
              total_words.append(w)

      # total word count after excluding words from stopwords class of NLTK
      word_count = len(total_words)

      # total count of personal nouns as indicated in the assignment
      personal_noun_count = count_pn(text)
      # total count of characters 
      count_char = 0
      # total count of characters 
      count_char = 0
      for w in total_words:
          count_char += len(w)
      average_word_length = count_char / len(total_words)

      # calculating positive score, negative score, polarity and subjectivity scores

      with open('negative-words.txt', 'r', encoding='latin-1') as neg_word:     # loading negative words
          n_words = neg_word.read()   

      with open('positive-words.txt', 'r', encoding='latin-1') as pos_word:     # loading positive words
          p_words = pos_word.read()

      words_after_stopwords_list = []

      for w in tokens:
          if w not in stop_words_list:
              words_after_stopwords_list.append(w)

      positive_words = nltk.word_tokenize(p_words)
      negative_words = nltk.word_tokenize(n_words)
      negative_score = 0
      positive_score = 0

      for word in words_after_stopwords_list:
          if word in negative_words:
              negative_score += 1
          elif word in positive_words:
              positive_score += 1

      # polarity score
      polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

      # subjectivity_score
      subjectivity_score = (positive_score + negative_score) / ((len(words_after_stopwords_list)) + 0.000001)

      complex_word_count = 0
      total_syllable_count = 0


      def syllable_count(word):                  #function to count syllables in a word
          word = word.lower()
          count = 0
          vowels = 'aeiou'
          for letter in word:
              if letter in vowels:
                  count += 1
          if word.endswith('es') or word.endswith('ed'):
              count -= 1
          return count


      def complex_word_checker(word):           #function to check if a word has more than 2 syllables
          if syllable_count(word) > 2:
              return True


      # calculation of total syllable count and complex word count

      for word in total_words:
          total_syllable_count += syllable_count(word)
          if complex_word_checker(word):
              complex_word_count += 1

      # calculation of percentae of complex words
      percentage_complex_words = complex_word_count / len(total_words)

      # calculation of syllable per word
      syllable_per_word = total_syllable_count / len(total_words)

      # calculating average sentence length 
      average_sentence_length = len(total_words) / len(sentences)
      

      # average no. of words per sentence
      average_number_of_words_per_sentence = len(total_words) / len(sentences)
      average_number_of_words_per_sentence = int(average_number_of_words_per_sentence)
      # calculation of fog index
      Fog_Index = 0.4 * (percentage_complex_words + average_sentence_length)         # calculating average sentence length 
      sheet.append([file_name,url,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_complex_words,Fog_Index,average_number_of_words_per_sentence,complex_word_count,word_count,syllable_per_word,personal_noun_count,average_word_length])


excel.save('Output_Data.xlsx')                           #saving the excel file 